In [1]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, start_date, end_date):
    """
    從 Yahoo Finance 下載指定股票或指數的歷史股價資料。

    Args:
        ticker (str): 股票代碼 (e.g., "2330.TW", "^TWII")。
        start_date (str): 資料起始日期 (YYYY-MM-DD)。
        end_date (str): 資料結束日期 (YYYY-MM-DD)。

    Returns:
        pd.DataFrame: 包含歷史股價的 DataFrame。
    """
    print(f"正在下載 {ticker} 從 {start_date} 到 {end_date} 的資料...")
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        print(f"錯誤：未能下載 {ticker} 的資料。請檢查代碼或日期範圍。")
        return pd.DataFrame()
    
    # 處理缺失值 (NaN)，通常是休市或資料不全導致
    # 這裡我們選擇直接刪除包含 NaN 的列，以確保後續計算的準確性
    initial_rows = len(data)
    data.dropna(inplace=True)
    if len(data) < initial_rows:
        print(f"警告：{ticker} 資料中存在缺失值，已刪除 {initial_rows - len(data)} 行。")
    
    # 確保 'Close' 欄位存在，這是計算技術指標的基礎
    if 'Close' not in data.columns:
        print(f"錯誤：{ticker} 資料中缺少 'Close' 欄位。")
        return pd.DataFrame()

    print(f"{ticker} 資料下載完成，共 {len(data)} 筆。")
    return data

if __name__ == "__main__":
    # 設定資料期間
    START_DATE = "2019-05-18" # 往前推五年
    END_DATE = "2024-05-17"   # 截至昨天

    # 下載台灣加權指數資料
    taiex_data = fetch_stock_data("^TWII", START_DATE, END_DATE)
    if not taiex_data.empty:
        # 可以將資料儲存為 CSV 檔案，方便後續讀取，避免重複下載
        taiex_data.to_csv("taiex_data.csv")
        print("台灣加權指數資料已儲存到 taiex_data.csv")

    # 下載台積電資料
    tsmc_data = fetch_stock_data("2330.TW", START_DATE, END_DATE)
    if not tsmc_data.empty:
        tsmc_data.to_csv("tsmc_data.csv")
        print("台積電資料已儲存到 tsmc_data.csv")

    print("\n資料獲取階段完成。")

正在下載 ^TWII 從 2019-05-18 到 2024-05-17 的資料...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^TWII']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


錯誤：未能下載 ^TWII 的資料。請檢查代碼或日期範圍。
正在下載 2330.TW 從 2019-05-18 到 2024-05-17 的資料...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['2330.TW']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


錯誤：未能下載 2330.TW 的資料。請檢查代碼或日期範圍。

資料獲取階段完成。


In [3]:
import pandas as pd
import ta # Technical Analysis Library
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean # 幾何平均數，用於計算年化報酬率

def calculate_indicators(data):
    """
    計算資料中的技術指標。
    Args:
        data (pd.DataFrame): 包含 'Close' 價格的 DataFrame。
    Returns:
        pd.DataFrame: 包含技術指標的 DataFrame。
    """
    # 計算移動平均線
    data['SMA20'] = ta.trend.sma_indicator(close=data['Close'], window=20)
    data['SMA60'] = ta.trend.sma_indicator(close=data['Close'], window=60)

    # 計算RSI
    data['RSI14'] = ta.momentum.rsi(close=data['Close'], window=14)

    # 計算MACD
    data['MACD'] = ta.trend.macd(close=data['Close'])
    data['MACD_Signal'] = ta.trend.macd_signal(close=data['Close'])
    data['MACD_Hist'] = ta.trend.macd_diff(close=data['Close'])
    
    # 刪除計算指標初期產生的 NaN 值
    data.dropna(inplace=True)
    return data

def ma_crossover_strategy(data, short_window=20, long_window=60):
    """
    移動平均線黃金交叉策略。
    Args:
        data (pd.DataFrame): 包含技術指標的 DataFrame。
        short_window (int): 短期均線期數。
        long_window (int): 長期均線期數。
    Returns:
        pd.DataFrame: 包含交易訊號和部位的 DataFrame。
    """
    df = data.copy()
    df['Signal'] = 0 # 0: 空手, 1: 買入
    
    # 產生買入訊號：短期均線由下往上穿越長期均線
    df.loc[(df[f'SMA{short_window}'] > df[f'SMA{long_window}']) & 
           (df[f'SMA{short_window}'].shift(1) <= df[f'SMA{long_window}'].shift(1)), 'Signal'] = 1

    # 產生賣出訊號：短期均線由上往下穿越長期均線
    df.loc[(df[f'SMA{short_window}'] < df[f'SMA{long_window}']) & 
           (df[f'SMA{short_window}'].shift(1) >= df[f'SMA{long_window}'].shift(1)), 'Signal'] = -1
    
    # 部位管理：將訊號轉換為實際部位 (-1: 做空, 0: 空手, 1: 做多)
    df['Position'] = 0
    # 我們假設每次買賣都是在訊號產生的第二天開盤執行 (實際回測通常這樣假設)
    # 這裡簡化為訊號產生當天收盤後執行，第二天開盤生效
    # df['Position'] = df['Signal'].replace(to_replace=0, method='ffill').shift(1) # 更複雜的部位管理需要循環
    
    # 簡單的部位持有邏輯：買入後一直持有，直到賣出訊號
    current_position = 0
    for i in range(len(df)):
        if df['Signal'].iloc[i] == 1: # 買入訊號
            current_position = 1
        elif df['Signal'].iloc[i] == -1: # 賣出訊號
            current_position = 0
        df.loc[df.index[i], 'Position'] = current_position
            
    # 第一天的部位通常為0，因為還沒有訊號
    df['Position'] = df['Position'].shift(1).fillna(0) # 昨天的部位決定今天的操作
    
    return df

def rsi_strategy(data, rsi_window=14, overbought_threshold=70, oversold_threshold=30):
    """
    RSI 超買超賣策略。
    Args:
        data (pd.DataFrame): 包含技術指標的 DataFrame。
        rsi_window (int): RSI 期數。
        overbought_threshold (int): 超買閾值。
        oversold_threshold (int): 超賣閾值。
    Returns:
        pd.DataFrame: 包含交易訊號和部位的 DataFrame。
    """
    df = data.copy()
    df['Signal'] = 0 # 0: 空手, 1: 買入
    
    # 產生買入訊號：RSI 低於超賣區
    df.loc[df[f'RSI{rsi_window}'] < oversold_threshold, 'Signal'] = 1

    # 產生賣出訊號：RSI 高於超買區
    df.loc[df[f'RSI{rsi_window}'] > overbought_threshold, 'Signal'] = -1

    # 部位管理 (同MA策略)
    current_position = 0
    for i in range(len(df)):
        if df['Signal'].iloc[i] == 1: # 買入訊號
            current_position = 1
        elif df['Signal'].iloc[i] == -1: # 賣出訊號
            current_position = 0
        df.loc[df.index[i], 'Position'] = current_position
            
    df['Position'] = df['Position'].shift(1).fillna(0)
    
    return df


def backtest_strategy(df, strategy_name="Strategy"):
    """
    回測策略並計算績效。
    Args:
        df (pd.DataFrame): 包含 'Close' 價格和 'Position' 部位的 DataFrame。
        strategy_name (str): 策略名稱，用於圖表標籤。
    Returns:
        dict: 包含各種績效指標的字典。
    """
    df['Daily_Return'] = df['Close'].pct_change()
    
    # 策略每日報酬率 (考慮昨天的持倉)
    df['Strategy_Daily_Return'] = df['Daily_Return'] * df['Position']

    # 計算累積報酬率
    df['Cumulative_Strategy_Return'] = (1 + df['Strategy_Daily_Return']).cumprod()
    df['Cumulative_Buy_Hold_Return'] = (1 + df['Daily_Return']).cumprod()

    # 初始化為1，因為cumprod會從第一個非NaN值開始計算
    df['Cumulative_Strategy_Return'].fillna(1, inplace=True)
    df['Cumulative_Buy_Hold_Return'].fillna(1, inplace=True)


    # --- 績效評估指標 ---
    performance = {}
    
    # 1. 總報酬率 (Total Return)
    performance['Total_Return'] = (df['Cumulative_Strategy_Return'].iloc[-1] - 1) * 100

    # 2. 年化報酬率 (Annualized Return)
    trading_days = len(df)
    years = trading_days / 252 # 假設每年約 252 個交易日
    performance['Annualized_Return'] = (df['Cumulative_Strategy_Return'].iloc[-1])**(1/years) - 1
    
    # 3. 最大回撤 (Maximum Drawdown, MDD)
    rolling_max = df['Cumulative_Strategy_Return'].cummax()
    daily_drawdown = (df['Cumulative_Strategy_Return'] / rolling_max) - 1
    performance['Max_Drawdown'] = daily_drawdown.min() * 100

    # 4. 夏普比率 (Sharpe Ratio)
    # 假設無風險利率為年化 1.5% (可根據實際情況調整)
    risk_free_rate_annual = 0.015
    risk_free_rate_daily = (1 + risk_free_rate_annual)**(1/252) - 1

    excess_daily_returns = df['Strategy_Daily_Return'] - risk_free_rate_daily
    sharpe_ratio = np.sqrt(252) * (excess_daily_returns.mean() / excess_daily_returns.std())
    performance['Sharpe_Ratio'] = sharpe_ratio

    # 5. 勝率 (Win Rate) 和 盈虧比 (Profit/Loss Ratio) - 需要更精確的交易紀錄
    # 這裡提供一個簡化的計算方式，基於每次產生訊號並改變部位來判斷
    
    # 找出所有交易點 (買入或賣出)
    trade_signals = df[df['Signal'] != 0].index
    
    num_wins = 0
    num_losses = 0
    total_profit = 0
    total_loss = 0
    
    # 簡單假設：每次買入後，直到下一次賣出或結算為一次交易
    in_position = False
    entry_price = 0
    
    for i in range(len(df)):
        current_date = df.index[i]
        
        # 處理買入訊號
        if df['Signal'].iloc[i] == 1 and not in_position:
            entry_price = df['Close'].iloc[i]
            in_position = True
        # 處理賣出訊號
        elif df['Signal'].iloc[i] == -1 and in_position:
            exit_price = df['Close'].iloc[i]
            profit_loss = (exit_price - entry_price) 
            
            if profit_loss > 0:
                num_wins += 1
                total_profit += profit_loss
            else:
                num_losses += 1
                total_loss += abs(profit_loss)
            in_position = False
    
    # 如果回測結束時仍然持倉，計算最後一次交易的損益
    if in_position:
        final_price = df['Close'].iloc[-1]
        profit_loss = (final_price - entry_price)
        if profit_loss > 0:
            num_wins += 1
            total_profit += profit_loss
        else:
            num_losses += 1
            total_loss += abs(profit_loss)

    performance['Win_Rate'] = (num_wins / (num_wins + num_losses)) * 100 if (num_wins + num_losses) > 0 else 0
    performance['Profit_Loss_Ratio'] = (total_profit / num_wins) / (total_loss / num_losses) if num_wins > 0 and num_losses > 0 else np.nan


    print(f"\n--- {strategy_name} 績效報告 ---")
    print(f"總報酬率: {performance['Total_Return']:.2f}%")
    print(f"年化報酬率: {performance['Annualized_Return']:.2%}")
    print(f"最大回撤: {performance['Max_Drawdown']:.2f}%")
    print(f"夏普比率: {performance['Sharpe_Ratio']:.2f}")
    print(f"勝率: {performance['Win_Rate']:.2f}%")
    print(f"盈虧比: {performance['Profit_Loss_Ratio']:.2f}")

    return performance, df


def plot_results(df, ticker, strategy_name, show_signals=True):
    """
    繪製策略權益曲線圖和股價與訊號圖。
    """
    # 權益曲線圖
    plt.figure(figsize=(14, 7))
    plt.plot(df['Cumulative_Strategy_Return'], label=f'{strategy_name} Equity Curve', color='blue')
    plt.plot(df['Cumulative_Buy_Hold_Return'], label='Buy and Hold Equity Curve', color='orange', linestyle='--')
    plt.title(f'{ticker} {strategy_name} vs. Buy and Hold Performance')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.show()

    # 股價與指標走勢圖 (可選，根據 show_signals 決定是否顯示買賣點)
    if show_signals:
        plt.figure(figsize=(14, 8))
        ax1 = plt.subplot(2, 1, 1) # 上半部分是股價和MA
        ax1.plot(df['Close'], label='Close Price', color='black', alpha=0.7)
        ax1.plot(df['SMA20'], label='SMA 20', color='red')
        ax1.plot(df['SMA60'], label='SMA 60', color='green')
        
        # 標示買入賣出點
        buy_points = df[df['Signal'] == 1]
        sell_points = df[df['Signal'] == -1]
        
        ax1.scatter(buy_points.index, buy_points['Close'], marker='^', color='lime', s=100, label='Buy Signal', alpha=1, zorder=5)
        ax1.scatter(sell_points.index, sell_points['Close'], marker='v', color='red', s=100, label='Sell Signal', alpha=1, zorder=5)
        
        ax1.set_title(f'{ticker} Close Price with MA Crossover Signals')
        ax1.set_ylabel('Price')
        ax1.legend()
        ax1.grid(True)

        ax2 = plt.subplot(2, 1, 2, sharex=ax1) # 下半部分是RSI
        ax2.plot(df['RSI14'], label='RSI 14', color='purple')
        ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
        ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
        ax2.set_title(f'{ticker} Relative Strength Index (RSI)')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('RSI Value')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()


if __name__ == "__main__":
    # --- 讀取資料 ---
    try:
        tsmc_data = pd.read_csv("tsmc_data.csv", index_col='Date', parse_dates=True)
        taiex_data = pd.read_csv("taiex_data.csv", index_col='Date', parse_dates=True)
    except FileNotFoundError:
        print("錯誤：找不到資料檔案。請先執行 stock_data_fetcher.py 下載資料。")
        exit()

    print("\n--- 開始處理台積電 (2330.TW) 資料 ---")
    tsmc_data_processed = calculate_indicators(tsmc_data.copy())

    # --- 執行 MA 黃金交叉策略回測 ---
    print("\n執行台積電 MA 黃金交叉策略 (20/60)...")
    tsmc_ma_strategy_df = ma_crossover_strategy(tsmc_data_processed.copy(), short_window=20, long_window=60)
    tsmc_ma_perf, tsmc_ma_results_df = backtest_strategy(tsmc_ma_strategy_df, "MA Crossover Strategy")
    plot_results(tsmc_ma_results_df, "2330.TW", "MA Crossover Strategy", show_signals=True) # 顯示買賣點

    # --- 執行 RSI 超買超賣策略回測 ---
    print("\n執行台積電 RSI 超買超賣策略 (14, 30/70)...")
    tsmc_rsi_strategy_df = rsi_strategy(tsmc_data_processed.copy(), rsi_window=14, overbought_threshold=70, oversold_threshold=30)
    tsmc_rsi_perf, tsmc_rsi_results_df = backtest_strategy(tsmc_rsi_strategy_df, "RSI Strategy")
    plot_results(tsmc_rsi_results_df, "2330.TW", "RSI Strategy", show_signals=False) # RSI策略，不顯示MA和MACD的訊號

    print("\n--- 開始處理台灣加權指數 (^TWII) 資料 ---")
    taiex_data_processed = calculate_indicators(taiex_data.copy())

    # --- 執行 MA 黃金交叉策略回測 (台灣加權指數) ---
    print("\n執行台灣加權指數 MA 黃金交叉策略 (20/60)...")
    taiex_ma_strategy_df = ma_crossover_strategy(taiex_data_processed.copy(), short_window=20, long_window=60)
    taiex_ma_perf, taiex_ma_results_df = backtest_strategy(taiex_ma_strategy_df, "MA Crossover Strategy")
    plot_results(taiex_ma_results_df, "^TWII", "MA Crossover Strategy", show_signals=True)

    # --- 執行 RSI 超買超賣策略回測 (台灣加權指數) ---
    print("\n執行台灣加權指數 RSI 超買超賣策略 (14, 30/70)...")
    taiex_rsi_strategy_df = rsi_strategy(taiex_data_processed.copy(), rsi_window=14, overbought_threshold=70, oversold_threshold=30)
    taiex_rsi_perf, taiex_rsi_results_df = backtest_strategy(taiex_rsi_strategy_df, "RSI Strategy")
    plot_results(taiex_rsi_results_df, "^TWII", "RSI Strategy", show_signals=False)

    print("\n所有策略回測與績效分析完成。")

ModuleNotFoundError: No module named 'ta'